In [3]:
from curl_cffi import requests  # 기존 requests 대신 사용
import re
import base64
import json
import pandas as pd
from datetime import datetime

def get_lme_aluminum_data():
    url = "https://en.macromicro.me/series/4257/aluminum-futures"

    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "cache-control": "no-cache",
        "pragma": "no-cache",
        "sec-ch-ua": '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
    }

    try:
        print("보안 차단을 우회하여 데이터를 불러오는 중입니다 (잠시만 기다려 주세요)...")

        # impersonate="chrome110" 설정을 통해 실제 크롬 브라우저의 네트워크 프로토콜을 흉내냅니다.
        response = requests.get(url, headers=headers, impersonate="chrome110")

        if response.status_code != 200:
            print(f"접속 실패: {response.status_code}")
            return

        pattern = r'atob\("([^"]+)"\)'
        matches = re.findall(pattern, response.text)

        if not matches:
            print("데이터 문자열을 찾을 수 없습니다. 페이지 소스를 확인해 보세요.")
            return

        encoded_data = max(matches, key=len)

        decoded_data = base64.b64decode(encoded_data).decode('utf-8')
        json_data = json.loads(decoded_data)

        df = pd.DataFrame(json_data, columns=['Date', 'Price'])

        df['Date'] = pd.to_datetime(df['Date'], unit='ms').dt.date

        print(f"\n성공! 총 {len(df)}행의 데이터를 추출했습니다.")
        print(f"최초 데이터 날짜: {df['Date'].min()}")
        print(f"최근 데이터 날짜: {df['Date'].max()}")

        df.to_excel("aluminum_price_full_history.xlsx", index=False)
        print("-" * 30)
        print("파일 저장 완료: aluminum_price_full_history.xlsx")

        return df

    except Exception as e:
        print(f"에러 발생: {str(e)}")

if __name__ == "__main__":
    get_lme_aluminum_data()

보안 차단을 우회하여 데이터를 불러오는 중입니다 (잠시만 기다려 주세요)...

성공! 총 9750행의 데이터를 추출했습니다.
최초 데이터 날짜: 1987-06-01
최근 데이터 날짜: 2026-01-21
------------------------------
파일 저장 완료: aluminum_price_full_history.xlsx


In [2]:
temp = get_lme_aluminum_data()

데이터를 불러오는 중입니다...
오류 발생: 403 Client Error: Forbidden for url: https://en.macromicro.me/series/4257/aluminum-futures
